# Monthly Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import TimeSeriesSplit


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout

from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys
from catboost import CatBoostRegressor

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972
FREQUENCY= "monthly"

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
kernel_rbf_noise = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),
            RandomForestRegressor(random_state=42, n_estimators=5),
            RandomForestRegressor(random_state=42, n_estimators=10),
            RandomForestRegressor(random_state=42, n_estimators=15), 
            SVR(kernel='rbf'),
            SVR(kernel='linear', max_iter=200),
            SVR(kernel='poly', max_iter=200),
            XGBRegressor(booster="gblinear",random_state=42, n_estimators=5, learning_rate=0.1),
            XGBRegressor(booster="gblinear",random_state=42, n_estimators=10, learning_rate=0.1),
            XGBRegressor(booster="gblinear",random_state=42, n_estimators=15, learning_rate=0.1),
            XGBRegressor(booster="gbtree",random_state=42, n_estimators=5, learning_rate=0.1),
            XGBRegressor(booster="gbtree", random_state=42, n_estimators=10, learning_rate=0.1),
            XGBRegressor(booster="gbtree", random_state=42, n_estimators=15, learning_rate=0.1),
            GaussianProcessRegressor(kernel=kernel_rbf_noise, random_state=42, n_restarts_optimizer=10),
            CatBoostRegressor(learning_rate=0.1, iterations=10, verbose=False,random_state=42),
            CatBoostRegressor(learning_rate=0.1, iterations=20, verbose=False,random_state=42),
            CatBoostRegressor(learning_rate=0.1, iterations=30, verbose=False,random_state=42),
            ]
name_regressors = ["Linear", "RF5", "RF10", "RF15", "SVR-rbf", "SVR-linear", "SVR-cubic","XGBL5", "XGBL10", "XGBL15", "XGBT5", "XGBT10", "XGBT15", "GPR-rbf-noise", "CBR-10", "CBR-20","CBR-30"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
assert len(regressors)==len(name_regressors)

## California

In [5]:
region="california"
metadata_path = f"data/climate_features/{region}/metadata.csv"
metadata = pd.read_csv(metadata_path)
metadata.reset_index(inplace=True, drop=True)
display(metadata)

,id,filename,season,indices
0,6e47cb06,predictor_6e47cb06_1.parquet,1,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
1,6e47cb06,predictor_6e47cb06_2.parquet,2,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
2,6e47cb06,predictor_6e47cb06_3.parquet,3,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
3,6e47cb06,predictor_6e47cb06_4.parquet,4,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
4,6e47cb06,predictor_6e47cb06_5.parquet,5,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
5,6e47cb06,predictor_6e47cb06_6.parquet,6,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
6,6e47cb06,predictor_6e47cb06_7.parquet,7,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
7,6e47cb06,predictor_6e47cb06_8.parquet,8,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
8,6e47cb06,predictor_6e47cb06_9.parquet,9,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...
9,6e47cb06,predictor_6e47cb06_10.parquet,10,df9a31c5-20a07cea-cfb03125-9169e0dc-0b0bffae-b...


In [6]:
results = pd.read_csv(f"data/climate_results/{region}_results/results.csv")
ids_results = results["id_data"].unique()
id_experiments = metadata["id"].unique()
ids_to_execute = [id for id in id_experiments if id not in ids_results]
print(len(ids_to_execute))

0


In [7]:
print(len(ids_results))
k=0
for id in ids_results:
    k+=1
    print("Executing",id, "iter", k)
    data = {i: pd.read_parquet(f"data/climate_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    rnn_model = Sequential([
    SimpleRNN(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(len(indices_of_interest))  # Predict 5 indices
    ])
    lstm_model = Sequential([
    LSTM(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(len(indices_of_interest))  # Predict 5 indices
    ])
    experiment_1 = PredictionExperiment(data, indices_of_interest, [rnn_model, lstm_model], ["RNN16", "LSTM16"], 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="TSCV", show=False)
    experiment_1.get_metrics("mape", stage="TSCV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/climate_results/{region}_results/results.csv")

2
Executing 6e47cb06 iter 1
1/1 [==============================] - 0s 17ms/step
Executing 5cb3fa02 iter 2
1/1 [==============================] - 0s 16ms/step


## Chile

In [8]:
region="chile"
metadata = pd.read_csv(f"data/climate_features/{region}/metadata.csv")
metadata.reset_index(inplace=True, drop=True)
display(metadata)

,id,filename,season,indices
0,978f49d7,predictor_978f49d7_1.parquet,1,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
1,978f49d7,predictor_978f49d7_2.parquet,2,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
2,978f49d7,predictor_978f49d7_3.parquet,3,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
3,978f49d7,predictor_978f49d7_4.parquet,4,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
4,978f49d7,predictor_978f49d7_5.parquet,5,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
...,...,...,...,...
223,8359c65d,predictor_8359c65d_8.parquet,8,422ed1de-59f835e4-c1c59e4d-03244181-1223f1f4-7...
224,8359c65d,predictor_8359c65d_9.parquet,9,422ed1de-59f835e4-c1c59e4d-03244181-1223f1f4-7...
225,8359c65d,predictor_8359c65d_10.parquet,10,422ed1de-59f835e4-c1c59e4d-03244181-1223f1f4-7...
226,8359c65d,predictor_8359c65d_11.parquet,11,422ed1de-59f835e4-c1c59e4d-03244181-1223f1f4-7...


In [9]:
results = pd.read_csv(f"data/climate_results/{region}_results/results.csv")
ids_results = results["id_data"].unique()
id_experiments = metadata["id"].unique()
ids_to_execute = [id for id in id_experiments if id not in ids_results]
print(len(ids_to_execute))


0


In [13]:
print(len(ids_results))
k=0
for id in ids_results[10:]:
    k+=1
    print("Executing",id, "iter", k)
    data = {i: pd.read_parquet(f"data/climate_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    rnn_model = Sequential([
    SimpleRNN(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(len(indices_of_interest))  # Predict 5 indices
    ])
    lstm_model = Sequential([
    LSTM(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(len(indices_of_interest))  # Predict 5 indices
    ])
    experiment_1 = PredictionExperiment(data, indices_of_interest, [rnn_model, lstm_model], ["RNN16", "LSTM16"], 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="TSCV", show=False)
    experiment_1.get_metrics("mape", stage="TSCV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/climate_results/{region}_results/results.csv")

20
Executing d7101242 iter 1
1/1 [==============================] - 0s 17ms/step
Executing 3832cbd6 iter 2
1/1 [==============================] - 0s 20ms/step
Executing 3df87a13 iter 3
1/1 [==============================] - 0s 18ms/step
Executing 30ab9bad iter 4
1/1 [==============================] - 0s 21ms/step
Executing 311dd366 iter 5
1/1 [==============================] - 0s 17ms/step
Executing 8c95fd00 iter 6
1/1 [==============================] - 0s 19ms/step
Executing 9f8163e4 iter 7
1/1 [==============================] - 0s 22ms/step
Executing 4effa258 iter 8
1/1 [==============================] - 0s 21ms/step
Executing 13a24caf iter 9
1/1 [==============================] - 0s 19ms/step
Executing 8359c65d iter 10
1/1 [==============================] - 0s 21ms/step
